In [1]:
#!pip install beautifulsoup4
#!pip install lxml

debug = True

In [40]:
import requests
from bs4 import BeautifulSoup
class Book:
    def __init__(self, title, ranking, url, price_ars):
        self.title = title
        self.ranking = ranking
        self.url = url
        self.price_ars = price_ars

URL = "https://www.cuspide.com/100-mas-vendidos"

try:
  response = requests.get(URL, timeout=5)
  response.raise_for_status()  # Esto lanzará una excepción si el código de estado no es 200
except requests.Timeout:
  print("La solicitud excedió el tiempo máximo de espera.")
except requests.RequestException as e:
  print(f"Hubo un error al hacer la solicitud: {e}")

soup = BeautifulSoup(response.text, 'html.parser')
book_items = soup.findAll('div',class_='col small-6 large-2 max-w-large')


libros = []
for book in book_items: 
  titulo = book.find('h3',class_='name product-title woocommerce-loop-product__title').text.strip()
  ranking = book.find('p',style='font-size: 12px; text-align: center; line-height: 1.3; margin-bottom: 10px; color: white; padding: 3px; background: #f26636; border-radius: 4px; margin-top: 5px').text.strip()
  url = book.find('a',href=True)['href']
  price_ars = book.find('span',class_='woocommerce-Price-amount amount').text.strip()
  # Añadimos la información a nuestra lista de libros
  book_data = {
        'title': titulo,
        'ranking': ranking,
        'url': url,
        'price_ars': price_ars
    }
  libros.append(book_data)



Number of libros: 89


In [44]:
import re
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import queue
import requests
urldolar = "https://dolarapi.com/v1/dolares/oficial"
urldolarblue = "https://dolarapi.com/v1/dolares/blue"
headers = {"Accept": "application/json"}
def get_usd_price():
    try:
        responsedolar = requests.get(urldolarblue, headers=headers, timeout=5)
        responsedolar.raise_for_status()
        datadolar = responsedolar.json()
        price_usd_blue = int(datadolar["venta"])
        return price_usd_blue
    except requests.Timeout:
        print("La solicitud excedió el tiempo máximo de espera.")
    except requests.RequestException as e:
        print(f"Hubo un error al hacer la solicitud: {e}")
    except KeyError:
        print("No se pudo obtener el precio del dólar blue.")
    return None


def get_usd_price_oficial():
    try:
        responsedolar = requests.get(urldolar, headers=headers, timeout=5)
        responsedolar.raise_for_status()
        datadolar = responsedolar.json()
        price_usd = int(datadolar["venta"])
        return price_usd
    except requests.Timeout:
        print("La solicitud excedió el tiempo máximo de espera.")
    except requests.RequestException as e:
        print(f"Hubo un error al hacer la solicitud: {e}")
    except KeyError:
        print("No se pudo obtener el precio del dólar blue.")
    return None

price_usd_blue = get_usd_price()
price_usd = get_usd_price_oficial()


# Función para limpiar y formatear el precio
def clean_price(price_str):
    price_str = re.sub(r'[^\d,]', '', price_str)
    price_float = float(price_str.replace(',', '.'))
    return round(price_float,2)


for libro in libros:
    cleaned_price = clean_price(libro['price_ars'])
    libro['price_ars'] = cleaned_price
    libro['price_usd'] = cleaned_price / price_usd
    libro['price_usd_blue'] = cleaned_price / price_usd_blue

'def fetch_price(libro, q):\n    try:\n        response = requests.get(libro[\'url\'], timeout=5)\n        response.raise_for_status()\n    except requests.Timeout:\n        print("La solicitud excedió el tiempo máximo de espera.")\n        q.put(1)\n        return\n    except requests.RequestException as e:\n        print(f"Hubo un error al hacer la solicitud: {e}")\n        q.put(1)\n        return\n\n    soup = BeautifulSoup(response.text, \'html.parser\')\n    precio_container = soup.find(\'div\', {\'id\': \'ctl00_ContentPlaceHolder1_rptFicha_ctl00_precioContainer\'})\n\n    if precio_container:\n        precios = precio_container.find_all(\'div\')\n        precio_pesos = clean_price(precios[0].text)\n        precio_dolares = clean_price(precios[1].text)\n    else:\n        precio_pesos = "Nada"\n        precio_dolares = "Nada"\n\n    # Agrega el precio en pesos y dólares al diccionario libro\n    libro[\'price_ars\'] = precio_pesos\n    libro[\'precio_dolares\'] = precio_dolares\n

In [45]:
if debug == True:
  #Imprimir el listado de libros con su precio
  for libro in libros:
          print(f"Libro: {libro['title']}")
          print("Precio en pesos:", libro['price_ars'])
          print("Precio en dólares:", libro['price_usd'])

Libro: ESTE DOLOR NO ES MIO
Precio en pesos: 9900.0
Precio en dólares: 27.049180327868854
Libro: TESIS SOBRE UNA DOMESTICACION
Precio en pesos: 9900.0
Precio en dólares: 27.049180327868854
Libro: EL VIENTO CONOCE MI NOMBRE
Precio en pesos: 10699.0
Precio en dólares: 29.23224043715847
Libro: BEYOND THE STORY ( EDICION EN ESPA/OL )
Precio en pesos: 7999.0
Precio en dólares: 21.8551912568306
Libro: EL VUELO DE LA LIBELULA
Precio en pesos: 8799.0
Precio en dólares: 24.040983606557376
Libro: UNA FAMILIA ANORMAL
Precio en pesos: 4199.0
Precio en dólares: 11.472677595628415
Libro: DESTROZA ESTE DIARIO ( A TODO COLOR )
Precio en pesos: 10200.0
Precio en dólares: 27.868852459016395
Libro: EL PODER DE LAS PALABRAS
Precio en pesos: 10599.0
Precio en dólares: 28.959016393442624
Libro: LAS INDIGNAS
Precio en pesos: 7499.0
Precio en dólares: 20.489071038251367
Libro: MIS DIAS EN LA LIBRERIA MORISAKI
Precio en pesos: 5850.0
Precio en dólares: 15.98360655737705
Libro: EL DUELO
Precio en pesos: 13100.0

In [5]:
#Cotización del dolar
import requests

urldolar = "https://dolarapi.com/v1/dolares/blue"
headers = {"Accept": "application/json"}
try:
  responsedolar = requests.get(urldolar,headers=headers, timeout=5)
  responsedolar.raise_for_status()
except responsedolar.Timeout:
  print("La solicitud excedió el tiempo máximo de espera.")
except responsedolar.RequestException as e:
  print(f"Hubo un error al hacer la solicitud: {e}")

datadolar = responsedolar.json()
precio_del_dolar_blue = int(datadolar["venta"])
if debug == True:
  print(precio_del_dolar_blue)

785


In [49]:
# Calculo de precio en dolar blue y fecha

from datetime import datetime

# Obtener la fecha actual
fecha_actual = datetime.now()

# Formatear la fecha al formato deseado, por ejemplo: "2023-09-21"
fecha_formateada = fecha_actual.strftime('%Y-%m-%d')

for libro in libros:
  libro['date'] = fecha_formateada

In [48]:
if debug == True:
  # Mostrar elementos antes de cargarlo en la base de datos
  for libro in libros:
      for key, value in libro.items():
          print(f"{key}: {value}")
      print("----------")  # Esto es solo para separar cada libro visualmente


title: ESTE DOLOR NO ES MIO
ranking: #1 ranking en venta
url: https://cuspide.com/producto/este-dolor-no-es-mio/
price_ars: 9900.0
price_usd: 27.049180327868854
price_usd_blue: 12.375
----------
title: TESIS SOBRE UNA DOMESTICACION
ranking: #2 ranking en venta
url: https://cuspide.com/producto/tesis-sobre-una-domesticacion/
price_ars: 9900.0
price_usd: 27.049180327868854
price_usd_blue: 12.375
----------
title: EL VIENTO CONOCE MI NOMBRE
ranking: #3 ranking en venta
url: https://cuspide.com/producto/el-viento-conoce-mi-nombre/
price_ars: 10699.0
price_usd: 29.23224043715847
price_usd_blue: 13.37375
----------
title: BEYOND THE STORY ( EDICION EN ESPA/OL )
ranking: #4 ranking en venta
url: https://cuspide.com/producto/beyond-the-story-edicion-en-espa-ol/
price_ars: 7999.0
price_usd: 21.8551912568306
price_usd_blue: 9.99875
----------
title: EL VUELO DE LA LIBELULA
ranking: #5 ranking en venta
url: https://cuspide.com/producto/el-vuelo-de-la-libelula/
price_ars: 8799.0
price_usd: 24.0409

In [46]:

for libro in libros:
    for key, value in libro.items():
        print(f"Key: {key}, Value: {value}, Type: {type(value)}")

Key: title, Value: ESTE DOLOR NO ES MIO, Type: <class 'str'>
Key: ranking, Value: #1 ranking en venta, Type: <class 'str'>
Key: url, Value: https://cuspide.com/producto/este-dolor-no-es-mio/, Type: <class 'str'>
Key: price_ars, Value: 9900.0, Type: <class 'float'>
Key: price_usd, Value: 27.049180327868854, Type: <class 'float'>
Key: price_usd_blue, Value: 12.375, Type: <class 'float'>
Key: title, Value: TESIS SOBRE UNA DOMESTICACION, Type: <class 'str'>
Key: ranking, Value: #2 ranking en venta, Type: <class 'str'>
Key: url, Value: https://cuspide.com/producto/tesis-sobre-una-domesticacion/, Type: <class 'str'>
Key: price_ars, Value: 9900.0, Type: <class 'float'>
Key: price_usd, Value: 27.049180327868854, Type: <class 'float'>
Key: price_usd_blue, Value: 12.375, Type: <class 'float'>
Key: title, Value: EL VIENTO CONOCE MI NOMBRE, Type: <class 'str'>
Key: ranking, Value: #3 ranking en venta, Type: <class 'str'>
Key: url, Value: https://cuspide.com/producto/el-viento-conoce-mi-nombre/, Ty

In [50]:
import pymysql

# Replace these values with your database credentials
host = '127.0.0.1'
port = 3306  # Default MySQL port
user = 'root'
password = 'root'
database = 'sys'

try:
    # Establish a connection to the MySQL server
    connection = pymysql.connect(
        host=host,
        port=port,
        user=user,
        password=password,
        database=database,
    )

    # Create a cursor object to interact with the database
    cursor = connection.cursor()

    cursor.execute('''
CREATE TABLE IF NOT EXISTS books (
    id INT AUTO_INCREMENT PRIMARY KEY,
    title VARCHAR(255),
    url VARCHAR(255),
    ranking VARCHAR(255),
    price_ars FLOAT,
    price_usd FLOAT,
    price_usd_blue FLOAT,
    date VARCHAR(255)
)
''')

    for libro in libros:
        cursor.execute('''
    INSERT INTO books (title, url, ranking, price_ars, price_usd, price_usd_blue, date)
       VALUES (%s, %s, %s, %s, %s, %s, %s)
    ''', (libro['title'], libro['url'], libro['ranking'], libro['price_ars'], libro['price_usd'], libro['price_usd_blue'], libro['date']))

    # Commit the changes to the database
    connection.commit()

    # Close the cursor and the connection
    cursor.close()
    connection.close()

except pymysql.Error as e:
    print(f"Error: {e}")
